## Imports

In [16]:
from numpy.random import seed
seed(888)
import tensorflow as tf
tf.random.set_seed(404)

In [1]:
import os
import numpy as np
import pandas as pd
from time import strftime
from PIL import Image

## Constants

In [2]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

test_skicar = 'test_number.png'

LOGGING_PATH = 'tensorboard_mnist_digit_logs'
NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_HEIGHT * IMAGE_WIDTH * CHANNELS

## Get the data

In [3]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

CPU times: total: 17.3 s
Wall time: 17.4 s


In [4]:
y_train_all.shape

(60000,)

In [5]:
y_train_all.shape

(60000,)

## Data preprocessing

In [6]:
# Re-scale 
x_train_all, x_test = x_train_all/ 255.0, x_test/ 255.0

#### Covert target values to one-hot encoding

In [7]:
values = y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [8]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [9]:
y_test = np.eye(NR_CLASSES)[y_test]

### Create validation dataset from training data

In [17]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [18]:
x_train.shape

(50000, 784)

### Setup Tensorflow Graph

In [19]:
tf.compat.v1.disable_eager_execution()
X = tf.compat.v1.placeholder(tf.float32, shape=[None, TOTAL_INPUTS])
Y = tf.compat.v1.placeholder(tf.float32, shape=[None, NR_CLASSES])

### NN Architecture
#### Hyperparameters

In [20]:
nr_epochs = 30
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

### Layer 1

In [21]:
initial_w1 = tf.compat.v1.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.1, seed=42)
w1 = tf.Variable(initial_value=initial_w1)

initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
b1 = tf.Variable(initial_value=initial_b1)

In [22]:
layer1_in = tf.matmul(X, w1) + b1
layer1_out = tf.nn.relu(layer1_in)

### Layer 2

In [23]:
initial_w2 = tf.compat.v1.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42)
w2 = tf.Variable(initial_value=initial_w2)

initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
b2 = tf.Variable(initial_value=initial_b2)

In [24]:
layer2_in = tf.matmul(layer1_out, w2) + b2
layer2_out = tf.nn.relu(layer2_in)

### Layer 3

In [25]:
initial_w3 = tf.compat.v1.truncated_normal(shape=[n_hidden2, NR_CLASSES], stddev=0.1, seed=42)
w3 = tf.Variable(initial_value=initial_w3)

initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES])
b3 = tf.Variable(initial_value=initial_b3)

In [26]:
layer3_in = tf.matmul(layer2_out, w3) + b3
output = tf.nn.softmax(layer3_in)

### Loss, Optimisation & Metrics

In [27]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output))

In [28]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(loss)

### Accuracy Metric

In [29]:
correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [30]:
tf.summary.scalar('accuracy', accuracy)
tf.summary.scalar('loss', loss)

<tf.Tensor 'loss/write_summary/Const:0' shape=() dtype=bool>

# Run Session

In [31]:
sess = tf.compat.v1.Session()

# Tensorboard Setup

In [32]:
# Folder for tensorboard
model_name = 'Model HW'

folder_name = f'{model_name} at {strftime("%H %M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Successfully created directory')

Successfully created directory


### Setup filewriter and merge summaries

In [33]:
merged_summary = tf.compat.v1.summary.merge_all()
train_writer = tf.compat.v1.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.compat.v1.summary.FileWriter(directory + '/validation')

In [34]:
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

In [35]:
b3.eval(sess)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

## Batching the data

In [36]:
size_of_batch = 1000
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [37]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size

    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
        
    end = index_in_epoch
        
    return data[start:end], labels[start:end]
    

In [38]:
tf.compat.v1.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [39]:
# for epoch in range(nr_epochs):
#     for i in range (nr_iterations):
#         batch_x, batch_y = next_batch(batch_size=size_of_batch,
#                                      data=x_train,
#                                      labels=y_train)
#         feed_dictionary = {X:batch_x, Y:batch_y}
#         sess.run(train_step, feed_dict=feed_dictionary)
#     s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
#     train_writer.add_summary(s, epoch)
#     print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
# print('Done training')

In [40]:
merged_summary = tf.compat.v1.summary.merge_all()
for epoch in range(nr_epochs):
    
    # ============= Training Dataset =========
    for i in range(nr_iterations):
        
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        sess.run(train_step, feed_dict=feed_dictionary)
        
    
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)  
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    # ================== Validation ======================
    
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)

print('Done training!')

Epoch 0 	| Training Accuracy = 0.859000027179718
Epoch 1 	| Training Accuracy = 0.8669999837875366
Epoch 2 	| Training Accuracy = 0.8709999918937683
Epoch 3 	| Training Accuracy = 0.875
Epoch 4 	| Training Accuracy = 0.8769999742507935
Epoch 5 	| Training Accuracy = 0.9629999995231628
Epoch 6 	| Training Accuracy = 0.9829999804496765
Epoch 7 	| Training Accuracy = 0.9860000014305115
Epoch 8 	| Training Accuracy = 0.984000027179718
Epoch 9 	| Training Accuracy = 0.9860000014305115
Epoch 10 	| Training Accuracy = 0.9869999885559082
Epoch 11 	| Training Accuracy = 0.9879999756813049
Epoch 12 	| Training Accuracy = 0.9890000224113464
Epoch 13 	| Training Accuracy = 0.9879999756813049
Epoch 14 	| Training Accuracy = 0.9890000224113464
Epoch 15 	| Training Accuracy = 0.9900000095367432
Epoch 16 	| Training Accuracy = 0.9909999966621399
Epoch 17 	| Training Accuracy = 0.9909999966621399
Epoch 18 	| Training Accuracy = 0.9909999966621399
Epoch 19 	| Training Accuracy = 0.9909999966621399
Epoch

## Make a prediction

In [41]:
img = Image.open('MNIST/test_img.png')

In [42]:
bw = img.convert('L')

In [43]:
img_array = np.invert(bw)

In [44]:
img_array.shape

(28, 28)

In [45]:
number = Image.open(test_skicar)
bw2 = number.convert('L')
img_array2 = np.invert(bw2)
test_img2 = img_array2.ravel()

In [46]:
test_img = img_array.ravel()

In [47]:
number = Image.open(test_skicar)
bw2 = number.convert('L')
img_array2 = np.invert(bw2)
test_img2 = img_array2.ravel()

In [48]:
test_img2.shape

(784,)

In [49]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X: [test_img2]})

In [50]:
number ## Test image

In [51]:
print(f'Prediction for test image is {prediction}')

Prediction for test image is [2]


# Reset for the next run

In [52]:
# train_writer.close()
# tf.compat.v1.reset_default_graph
# validation_writer.close()
# sess.close()